<a href="https://colab.research.google.com/github/HansHenseler/masdav2022/blob/main/Part_7_Exercises_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing and visualising geolocation data

Part 7

Master of Advanced Studies in Digital Forensics & Cyber Investigation

Data Analytics and Visualization for Digital Forensics

(c) Hans Henseler, 2022

# Part 7

Importing libraries

In [2]:
# Need to install various packages. 
#
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install geopandas
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (4,094 kB/s)
Selecting previously unselected

In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
import folium
import plotly_express as px

In [4]:
!sudo apt-get install -y exiftool

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libimage-exiftool-perl' instead of 'exiftool'
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libarchive-zip-perl libmime-charset-perl libposix-strptime-perl libsombok3
  libunicode-linebreak-perl
Suggested packages:
  libencode-hanextra-perl libpod2-base-perl
The following NEW packages will be installed:
  libarchive-zip-perl libimage-exiftool-perl libmime-charset-perl
  libposix-strptime-perl libsombok3 libunicode-linebreak-perl
0 upgraded, 6 newly installed, 0 to remove and 20 not upgraded.
Need to get 3,395 kB of archives.
After this operation, 20.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libarchive-zip-perl all 1.60-1ubuntu0.1 [84.6 kB]
Get:2 http://archive.ub

# Exercises

## 1 Analyse pictures from an Android 10 phone for geotags and plot on a map

In [5]:
# Download Android_10.zip from Digital Corpora (takes app 7 minutes to download in Google Colab)
#
!wget "https://digitalcorpora.s3.amazonaws.com/corpora/mobile/android_10/Android_10.zip"

--2022-08-26 08:52:11--  https://digitalcorpora.s3.amazonaws.com/corpora/mobile/android_10/Android_10.zip
Resolving digitalcorpora.s3.amazonaws.com (digitalcorpora.s3.amazonaws.com)... 52.218.181.195
Connecting to digitalcorpora.s3.amazonaws.com (digitalcorpora.s3.amazonaws.com)|52.218.181.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10476724716 (9.8G) [application/zip]
Saving to: ‘Android_10.zip’

Android_10.zip      100%[===================>]   9.76G  26.0MB/s    in 6m 28s  

2022-08-26 08:58:39 (25.8 MB/s) - ‘Android_10.zip’ saved [10476724716/10476724716]



In [6]:
# In this zip there is a zip file containing a logical extraction of the file system
# /Android 10 Image with Documentation/Non-Cellebrite Extraction/Pixel 3.zip
# extract this zip file (takes about 4 minutes, use -j to ignore file path)

!unzip -j Android_10.zip  Android\ 10\ Image\ with\ Documentation/Non-Cellebrite\ Extraction/Pixel\ 3.zip

Archive:  Android_10.zip
  inflating: Pixel 3.zip             


In [7]:
# investigate the contents of this zip file for jpg files, extract, run the exiftool and plot on a map
# hint: use the unzip -l option (e.g. to find *.jpg)

!unzip -l "Pixel 3.zip" | grep jpg

    22062  2020-02-08 14:54   Pixel 3/data/data/com.enflick.android.TextNow/app_Leanplum_Documents/InApp_Countdown_Timer (2).jpg
    15012  2020-02-08 14:54   Pixel 3/data/data/com.enflick.android.TextNow/app_Leanplum_Documents/nonsale_avatar.jpg
   135034  2020-02-08 14:54   Pixel 3/data/data/com.enflick.android.TextNow/app_Leanplum_Documents/nonsale_defaultinterstitial_extra101.jpg
    41336  2020-01-31 21:15   Pixel 3/data/data/com.google.android.googlequicksearchbox/files/recently/thisisdfir@gmail.com-3692132869510698294.jpg
    13696  2020-01-31 21:14   Pixel 3/data/data/com.google.android.googlequicksearchbox/files/recently/thisisdfir@gmail.com-5812264224591469663.jpg
    29512  2020-01-31 21:13   Pixel 3/data/data/com.google.android.googlequicksearchbox/files/recently/thisisdfir@gmail.com-6982875141504110754.jpg
   577563  2020-01-29 11:55   Pixel 3/data/data/com.google.vr.apps.ornament/app_preview_temp/shared/com.google.vr.apps.ornament.content.stickers.spark/spark_preview.jpg


In [8]:
# create a folder images in your colab (manually). Export all *.jpg and ignor paths (-j option)
# to this images folder (-d ./images). Note: choose N for None if unzip asks if it must overwrite a file.
#
!unzip -j -d ./images "Pixel 3.zip" "*.jpg"

Archive:  Pixel 3.zip
  inflating: ./images/InApp_Countdown_Timer (2).jpg  
  inflating: ./images/nonsale_avatar.jpg  
  inflating: ./images/nonsale_defaultinterstitial_extra101.jpg  
  inflating: ./images/thisisdfir@gmail.com-3692132869510698294.jpg  
  inflating: ./images/thisisdfir@gmail.com-5812264224591469663.jpg  
  inflating: ./images/thisisdfir@gmail.com-6982875141504110754.jpg  
  inflating: ./images/spark_preview.jpg  
 extracting: ./images/6BIqfFk.jpg    
 extracting: ./images/CXW0PEe.jpg    
  inflating: ./images/82150035d650b73619f729f8797a2979.jpg  
  inflating: ./images/043a8097f43c6fb006dd4a97ee9447be.jpg  
  inflating: ./images/04a860c62dd3c32bc987962bf50826f0.jpg  
  inflating: ./images/0cf03778a47919ffa242abbb61045546.jpg  
  inflating: ./images/15093b05575b105cfb87ca1f977ba155.jpg  
  inflating: ./images/235cf218066aa4e50fab6637df280536.jpg  
  inflating: ./images/2f8011301fcab6a4e63fd3d3f58bdf2e.jpg  
  inflating: ./images/40a02bf1a684b89c672336492d738f56.jpg  
  i

In [9]:
!exiftool -csv -c '%+.6f' -gpslatitude -gpslongitude -GPSLatitudeRef -GPSLongitudeRef -r -ext jpg -T ./images > data.csv

Error: File format error - ./images/._thumbnail.jpg


In [10]:
# we can ignore the file format error
#
!cat data.csv

SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef
./images/295.jpg,-,-,-,-
./images/b372c084a7a80b4fe4d6906257b54098.jpg,-,-,-,-
./images/1-2116fb71f259777d12e7d9ff176946a3.jpg,-,-,-,-
./images/40a02bf1a684b89c672336492d738f56.jpg,-,-,-,-
./images/04a860c62dd3c32bc987962bf50826f0.jpg,-,-,-,-
./images/IMG_20200203_213026.jpg,-,-,-,-
./images/share-bg.321973c9.jpg,-,-,-,-
./images/7d6a48ef8e6f9662983bbe8d8989d721.jpg,-,-,-,-
./images/86.jpg,-,-,-,-
./images/r1920_10000999_cr_tail.jpg,-,-,-,-
./images/235cf218066aa4e50fab6637df280536.jpg,-,-,-,-
./images/6-1ebca57f4b6b3b177765a248a462b476.jpg,-,-,-,-
./images/27.jpg,-,-,-,-
./images/28.jpg,-,-,-,-
./images/d1885b2f147f9a3fee77c9d739f7231b.jpg,-,-,-,-
./images/87.jpg,-,-,-,-
./images/62ec3f716d700c3cb4d474345f46652f.jpg,-,-,-,-
./images/IMG_20200203_213034.jpg,-,-,-,-
./images/62.jpg,-,-,-,-
./images/101.jpg,-,-,-,-
./images/IMG-d63d12a7fc6ad79a3b6116d9054eb217-V.jpg,-,-,-,-
./images/65.jpg,-,-,-,-
./images/294_reduced.jpg

In [25]:
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SourceFile       131 non-null    object
 1   GPSLatitude      131 non-null    object
 2   GPSLongitude     131 non-null    object
 3   GPSLatitudeRef   131 non-null    object
 4   GPSLongitudeRef  131 non-null    object
dtypes: object(5)
memory usage: 5.2+ KB


In [26]:
df

,SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef
0,./images/295.jpg,-,-,-,-
1,./images/b372c084a7a80b4fe4d6906257b54098.jpg,-,-,-,-
2,./images/1-2116fb71f259777d12e7d9ff176946a3.jpg,-,-,-,-
3,./images/40a02bf1a684b89c672336492d738f56.jpg,-,-,-,-
4,./images/04a860c62dd3c32bc987962bf50826f0.jpg,-,-,-,-
...,...,...,...,...,...
126,./images/CXW0PEe.jpg,-,-,-,-
127,./images/rzA2EiW.jpg,-,-,-,-
128,./images/640px-Enigma_(crittografia)_-_Museo_s...,-,-,-,-
129,./images/cover-8544a5b2e95efd056b853be1907b60a...,-,-,-,-


In [27]:
df = df[df.GPSLatitude !=  '-'].reset_index()
df

,index,SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef
0,81,./images/IMG_20200209_132353.jpg,+35.660981,-78.879897,North,West
1,84,./images/IMG_20190917_082637.jpg,+35.735647,-78.636367,North,West
2,113,./images/IMG_20200213_195030.jpg,+35.658867,-78.874217,North,West


In [28]:
jpg_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.GPSLatitude, df.GPSLongitude))
jpg_gdf

,index,SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef,geometry
0,81,./images/IMG_20200209_132353.jpg,+35.660981,-78.879897,North,West,POINT (35.66098 -78.87990)
1,84,./images/IMG_20190917_082637.jpg,+35.735647,-78.636367,North,West,POINT (35.73565 -78.63637)
2,113,./images/IMG_20200213_195030.jpg,+35.658867,-78.874217,North,West,POINT (35.65887 -78.87422)


In [29]:
map = folium.Map(location = [35.660981,-78.879897,], tiles='OpenStreetMap' , width='75%',height='75%',zoom_start = 8)

# Create a geometry list from the GeoDataFrame
jpg_list = [[point.xy[0][0], point.xy[1][0]] for point in jpg_gdf.geometry ]

# Iterate through list and add a marker for each host in jpg_gdf

i = 0
for coordinates in jpg_list:
    map.add_child(folium.Marker(location = coordinates, popup=jpg_gdf.SourceFile[i]))
    i = i+1

map

## 2 Can you find any other sources besides jpg files that contain geo location and can you also extract  and visualise that information?

hint: consider sqlite files from Android apps

In [16]:
# One sqlite database is the myplaces db from Google maps
#
# gmm_myplaces.db
#

!unzip -l "Pixel 3.zip" | grep gmm_myplaces

    40960  2020-02-09 14:45   Pixel 3/data/data/com.google.android.apps.maps/databases/gmm_myplaces.db
    40960  2020-02-09 14:45   Pixel 3/sbin/.magisk/mirror/data/data/com.google.android.apps.maps/databases/gmm_myplaces.db


In [17]:
!unzip -j "Pixel 3.zip"  "Pixel 3/sbin/.magisk/mirror/data/data/com.google.android.apps.maps/databases/gmm_myplaces.db"

Archive:  Pixel 3.zip
  inflating: gmm_myplaces.db         


In [18]:
import sqlite3

gmmdb = sqlite3.connect('gmm_myplaces.db')

In [19]:
df = pd.read_sql_query("SELECT * from sync_item", gmmdb)
df

,corpus,key_string,timestamp,merge_key,feature_fprint,latitude,longitude,is_local,sync_item
0,8,1:0,1554407343604,0,None,35734602,-78636654,0,b'\x12\x031:02\xd1\x01\n\x04\x08\x01\x10\x00\x...


In [20]:
# need to divide by 1000000 because there is no decimal point

gmmdb_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.latitude/1000000, df.longitude/1000000))
gmmdb_gdf

,corpus,key_string,timestamp,merge_key,feature_fprint,latitude,longitude,is_local,sync_item,geometry
0,8,1:0,1554407343604,0,None,35734602,-78636654,0,b'\x12\x031:02\xd1\x01\n\x04\x08\x01\x10\x00\x...,POINT (35.73460 -78.63665)


In [30]:
# the google timestamp is unix epoch in milliseconds. If we /1000 we can use datetime.fromtimestamp to get a datetime

from datetime import datetime
print(datetime.fromtimestamp(gmmdb_gdf.timestamp[0]/1000).strftime('Google %Y-%m-%d %H:%M:%S'))

Google 2019-04-04 19:49:03


In [22]:

map = folium.Map(location = [gmmdb_gdf.geometry[0].xy[0][0],gmmdb_gdf.geometry[0].xy[1][0]], tiles='OpenStreetMap' , width='75%',height='75%',zoom_start = 8)

# Create a geometry list from the GeoDataFrame
gmmdb_list = [[point.xy[0][0], point.xy[1][0]] for point in gmmdb_gdf.geometry ]

# Iterate through list and add a marker for each host in jpg_gdf
i = 0
for coordinates in gmmdb_list:
    popup = datetime.fromtimestamp(gmmdb_gdf.timestamp[i]/1000).strftime('Google myplaces timestamp %Y-%m-%d %H:%M:%S')
    map.add_child(folium.Marker(location = coordinates, popup=popup))
    i = i+1
map

In [31]:
# this location appears to be near the location of picture ./images/IMG_20190917_082637.jpg
# what does the exiftool tell us?

!exiftool ./images/IMG_20190917_082637.jpg

ExifTool Version Number         : 10.80
File Name                       : IMG_20190917_082637.jpg
Directory                       : ./images
File Size                       : 5.5 MB
File Modification Date/Time     : 2020:01:31 21:22:58+00:00
File Access Date/Time           : 2022:08:26 09:05:32+00:00
File Inode Change Date/Time     : 2022:08:26 09:05:21+00:00
File Permissions                : rw-r--r--
File Type                       : JPEG
File Type Extension             : jpg
MIME Type                       : image/jpeg
Exif Byte Order                 : Little-endian (Intel, II)
Make                            : Google
Camera Model Name               : Pixel 3
Orientation                     : Rotate 90 CW
X Resolution                    : 72
Y Resolution                    : 72
Resolution Unit                 : inches
Software                        : HDR+ 1.0.265696846zdh
Modify Date                     : 2019:09:17 08:26:37
Y Cb Cr Positioning             : Centered
Exposure Time 

In [24]:
# any other places? Here is a pdf file with a chapter on Android en Geo locations
# https://content-calpoly-edu.s3.amazonaws.com/cci/1/documents/ccic_forensics_manual/Android%20Forensics%20Chapter%209%20-%20Location%20Data.pdf
# but it could be an old manual

!unzip -l "Pixel 3.zip" | grep networkhistory